In [1]:
from dgp import DGP_Finite
from inference import Inference2, Inference
import numpy as np
import pandas as pd 

# generate multuple sample size, multiple model designs, multiple inference method,
# a single estimator
dgp = DGP_Finite('1',1000)
Yobs, D, A = dgp.get_data()
inf = Inference2(Yobs, D, A, dgp.cluster, dgp.tuple_idx, dgp.tau10)
print(inf.inference('mp'), inf.inference('robust'), inf.inference('clustered'))

def cover_rate(sample_size=1000, modelY='1', ntrials=2000):
    cover = np.zeros((ntrials, 3))
    cf_length = np.zeros((ntrials, 3))
    for i in range(2000):
        dgp = DGP_Finite(modelY, sample_size)
        Yobs, D, A = dgp.get_data()
        inf = Inference2(Yobs, D, A, dgp.cluster, dgp.tuple_idx, dgp.tau10)
        cover[i,0] = inf.inference('mp')
        cf_length[i,0] = inf.se_tau10*19.6*2
        cover[i,1] = inf.inference('robust')
        cf_length[i,1] = inf.se_tau10*19.6*2
        cover[i,2] = inf.inference('clustered')
        cf_length[i,2] = inf.se_tau10*19.6*2
    return np.mean(cover, axis=0), np.mean(cf_length, axis=0)


def cover_rate_finite(sample_size=1000, modelY='1', ntrials=2000):
    cover = np.zeros((ntrials, 3))
    cf_length = np.zeros((ntrials, 3))
    dgp = DGP_Finite(modelY, sample_size)
    for i in range(2000):
        Yobs, D, A = dgp.get_data()
        inf = Inference2(Yobs, D, A, dgp.cluster, dgp.tuple_idx, dgp.tau10)
        cover[i,0] = inf.inference('mp')
        cf_length[i,0] = inf.se_tau10*19.6*2
        cover[i,1] = inf.inference('robust')
        cf_length[i,1] = inf.se_tau10*19.6*2
        cover[i,2] = inf.inference('clustered')
        cf_length[i,2] = inf.se_tau10*19.6*2
    return np.mean(cover, axis=0), np.mean(cf_length, axis=0)

1 1 1


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/apply.py:290: UserWarning: `x` isn't a recognized object; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  results[i] = self.f(v)


In [3]:
from joblib import Parallel, delayed
import multiprocessing

modelYs = ['1','2','3','4','5','6']
sample_sizes = [50, 100, 200, 500, 1000]

sample_sizes = [40, 80, 160, 480, 1000]

qk_pairs = [(q,k) for q in modelYs for k in sample_sizes]
def processInput(qk):
    q, k = qk
    cover, cf = cover_rate(k, q)
    return (q,k,cover,cf)
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in qk_pairs)
output = np.zeros((len(modelYs)*3,len(sample_sizes)))
cf_output = np.zeros((len(modelYs)*3,len(sample_sizes)))
for (q,k,cover,cf) in results:
    i = int(q)-1
    j = sample_sizes.index(k)
    output[i*3:i*3+3,j] = cover
    cf_output[i*3:i*3+3,j] = cf

output = np.zeros((len(modelYs)*3*2,len(sample_sizes)))
for (q,k,cover,cf) in results:
    i = int(q)-1
    j = sample_sizes.index(k)
    out = [cover[0], cf[0], cover[1], cf[1], cover[2], cf[2]]
    output[i*6:i*6+6,j] = out
print(output)
pd.DataFrame(output).to_csv("finite_pop_cover_rate_cf.csv")

[[ 2.09059733  1.29988532  0.88448906  0.50540412  0.35053067]
 [ 5.23699319  3.7559094   2.65925572  1.54373362  1.07086228]
 [ 1.88945522  1.27257969  0.88572448  0.50702789  0.35088164]
 [ 5.03755465  3.43847842  2.40605648  1.38579312  0.95999242]
 [ 7.43937286  5.3316549   3.77710926  2.18493765  1.5149678 ]
 [ 9.78851794  6.70140252  4.64574453  2.64675303  1.82744625]
 [ 4.63943578  3.19280445  2.24160142  1.29275434  0.89540124]
 [ 6.78102924  4.85952735  3.45384519  1.99975768  1.38621195]
 [ 9.02353294  6.17612772  4.29313105  2.44892327  1.69085264]
 [ 4.66767834  3.18989249  2.24252315  1.29276614  0.8962576 ]
 [ 6.33686301  4.52819296  3.22283598  1.86563087  1.29437254]
 [ 9.04884341  6.16725645  4.2942263   2.44887235  1.69246087]
 [19.34535669 10.86693211  6.07844366  2.56419598  1.54149014]
 [25.20717721 18.52031235 13.32864435  7.7258967   5.35501551]
 [18.38784834 11.36629217  7.25818919  3.79970228  2.55173925]
 [ 5.19003935  3.57732239  2.50334612  1.44709115  1.00